#### 모듈

In [5]:
import os
import os.path as path

import pandas as pd

from sklearn.preprocessing import LabelEncoder
import numpy as np

#### 인코더 불러오기

In [6]:
# path
BasePath = "./encoders"

# get list of encoder files
encoder_file_list = os.listdir(BasePath)

# print list of encoder files
print('encoder_file_list :', encoder_file_list)

# load encoder files
encoder_dict = {}

for file in encoder_file_list:
    encoder_name = file.replace(".npy", "")
    encoder_dict[encoder_name] = LabelEncoder()
    classes = np.load(path.join(BasePath, file), allow_pickle=True)
    encoder_dict[encoder_name].classes_ = np.load(path.join(BasePath, file), allow_pickle=True)
    
    if encoder_name == "os_version":
        encoder_dict[encoder_name].fit(list(map(lambda x: str(x), classes)))

# encoder modify
for encoder_name in encoder_dict.keys():
    # reorder encoder : make encoder[0] = nan
    encoder_dict[encoder_name].classes_ = \
        np.append(encoder_dict[encoder_name].classes_[-1],\
                    encoder_dict[encoder_name].classes_[:-1])

    print(f"{encoder_name} : {encoder_dict[encoder_name].classes_[:5]}")

# reorder 'event_type' encoder classes
encoder_dict['event_type'].classes_ = np.array([
    'error',
    'enter.main_page',
    'enter.signup_page',
    'complete.signup',
    'enter.content_page',
    'click.content_page_start_content_button',
    'click.content_page_more_review_button',
    'enter.payment_page',
    'complete.subscription',
    'renew.subscription',
    'resubscribe.subscription',
    'start.free_trial',
    'start.content',
    'enter.lesson_page',
    'complete.lesson',
    'click.lesson_page_related_question_box',
    'end.content',
    'click.cancel_plan_button'
])
# encoder test
print(encoder_dict["event_type"].classes_[:5])


encoder_file_list : ['button.name.npy', 'button_name.npy', 'city.npy', 'content.difficulty.npy', 'content.id.npy', 'country.npy', 'coupon.discount_amount.npy', 'device_family.npy', 'device_type.npy', 'event_type.npy', 'is_free_trial.npy', 'is_trial.npy', 'language.npy', 'lesson.id.npy', 'os_name.npy', 'os_version.npy', 'paid_amount.npy', 'pg.type.npy', 'plan.price.npy', 'plan.type.npy', 'platform.npy', 'question.id.npy', 'trial.type.npy', 'type.npy', 'user_id.npy']
button.name : [nan '구독 시작하기' '무료로 들어보기' '무료로 시작하기' '최저가 혜택 받기']
button_name : [nan '무료로 시작하기' '수강시작' '수강하기' '이어하기']
city : [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
content.difficulty : [nan 'advanced' 'beginner' 'hard' 'intermediate']
content.id : [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']
country : [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
coupon.discount_amount : [nan -52400 -48400 0 478]

#### 인코더 테스트

In [7]:
## 주의사항
# 
# 1. BaseFilePath에 데이터 파일 경로를 설정해주세요.
# 2. 제외하고 싶은 컬럼이 있으면 exclude_columns에 추가해주세요.
# 3. 인코딩을 하고 컬럼 이름을 바꿔 주세요.

## 함수 설명
# 
# .transform()을 이용해서 데이터를 변환해주세요.
# 
# 데이터[칼럼] = encoder_dict[칼럼].transform(데이터[칼럼])
# 예시 : df[col] = encoder_dict[col].transform(df[col])
# 
# 
# .inverse_transform()을 이용해서 데이터를 역변환해주세요.
# 
# 데이터[칼럼] = encoder_dict[칼럼].inverse_transform(데이터[칼럼])
# 예시 : df[col] = encoder_dict[col].inverse_transform(df[col])


## 인코딩

# 개인적으로 BaseFilePath를 설정해주었는데, 이를 이용해서 파일 리스트를 불러오는 코드입니다.
BaseFilePath = 'd:\Codeit\intermediate_project\주제 2. 구독서비스 프로덕트 데이터 분석'
filelist = os.listdir(BaseFilePath)[:-1]

# 제외 컬럼
exclude_columns = ['device_carrier', 'os_version']

# 제외 컬럼 : 제외하고 싶은 컬럼을 추가합니다.
exclude_columns_plus = []

# display file list
print(filelist)

# load data
dfs = []

for file in filelist:
    if file.endswith('.csv'):
        df = pd.read_csv(path.join(BaseFilePath, file), nrows=200)
        
        # preprocess
        # convert os_version to string
        df['os_version'] = df['os_version'].astype(str)
        # drop 
        df = df.drop(exclude_columns + exclude_columns_plus, axis=1)
        # convert client_event_time to datetime
        df['client_event_time'] = pd.to_datetime(df['client_event_time'])
        
        dfs.append(df)
        print(f"file: {file}, shape: {df.shape}")

for i, df in enumerate(dfs):
    print(f"file: {filelist[i]} encodings - ")
    
    # encode categorical columns using the loaded encoders
    for col in df.columns:
        # exception handling
        if col not in encoder_dict:
            continue
        if col in exclude_columns + exclude_columns_plus:
            continue
        
        # encode using the loaded encoder
        df[col] = encoder_dict[col].transform(df[col])
        print(f"{col}: {encoder_dict[col].classes_[:5]}")
    
    # display result head
    display(df.head())

['click.cancel_plan_button.csv', 'click.content_page_more_review_button.csv', 'click.content_page_start_content_button.csv', 'click.lesson_page_related_question_box.csv', 'complete.lesson.csv', 'complete.signup.csv', 'complete.subscription.csv', 'end.content.csv', 'enter.content_page.csv', 'enter.lesson_page-002.csv', 'enter.main_page.csv', 'enter.payment_page.csv', 'enter.signup_page.csv', 'renew.subscription.csv', 'resubscribe.subscription.csv', 'start.content.csv', 'start.free_trial.csv']
file: click.cancel_plan_button.csv, shape: (200, 10)
file: click.content_page_more_review_button.csv, shape: (200, 11)
file: click.content_page_start_content_button.csv, shape: (200, 13)


<>:24: SyntaxWarning: invalid escape sequence '\C'
<>:24: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_10860\1609756926.py:24: SyntaxWarning: invalid escape sequence '\C'
  BaseFilePath = 'd:\Codeit\intermediate_project\주제 2. 구독서비스 프로덕트 데이터 분석'


file: click.lesson_page_related_question_box.csv, shape: (200, 13)
file: complete.lesson.csv, shape: (200, 12)
file: complete.signup.csv, shape: (200, 11)
file: complete.subscription.csv, shape: (200, 14)
file: end.content.csv, shape: (200, 11)
file: enter.content_page.csv, shape: (200, 11)
file: enter.lesson_page-002.csv, shape: (200, 14)
file: enter.main_page.csv, shape: (200, 10)
file: enter.payment_page.csv, shape: (200, 10)
file: enter.signup_page.csv, shape: (200, 10)
file: renew.subscription.csv, shape: (200, 14)
file: resubscribe.subscription.csv, shape: (200, 14)
file: start.content.csv, shape: (200, 12)
file: start.free_trial.csv, shape: (200, 13)
file: click.cancel_plan_button.csv encodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'And

,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id
0,886,2023-04-10 14:57:48.556,158,1029,609,17,38,4,1,111466
1,3440,2023-03-12 08:26:16.224,158,130,5,17,15,23,1,55652
2,3945,2023-03-12 08:23:04.955,158,1029,609,17,38,4,1,128371
3,967,2023-03-12 08:32:39.271,158,1029,609,17,38,4,1,45697
4,1260,2023-04-12 11:36:28.311,158,1029,609,17,38,4,1,133511


file: click.content_page_more_review_button.csv encodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
content.id: [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']


...

,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,plan.price,paid_amount,coupon.discount_amount,pg.type
0,888,2023-04-05 21:11:45.525,158,1029,609,10,38,10,1,46100,2,13,3,0
1,3440,2023-06-30 09:59:56.848,158,773,488,10,38,7,1,73557,2,13,3,0
2,2765,2023-06-13 06:20:22.524,158,1029,609,10,38,4,1,55369,2,13,3,0
3,3694,2023-06-11 11:02:11.878,158,1029,609,10,38,4,1,174991,12,50,3,0
4,1996,2023-06-13 14:39:25.736,85,135,38,10,33,4,1,24351,12,50,3,0


file: start.content.csv encodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
content.id: [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']
content.difficulty: [nan 'advanced' 'beginner' 'hard' 'intermediate']


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,content.id,content.difficulty
0,888,2023-04-05 21:14:39.906,158,1029,609,12,38,10,1,46100,57,2
1,3440,2023-04-05 21:50:19.674,158,570,158,12,38,34,1,4752,201,2
2,3440,2023-04-10 14:06:26.585,158,570,158,12,38,40,1,86684,85,2
3,3881,2023-04-10 14:15:29.040,31,1029,609,12,15,4,1,165440,85,2
4,3440,2023-04-10 14:21:36.467,158,1029,609,12,38,4,1,94157,22,1


file: start.free_trial.csv encodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
plan.price: [nan 14328 15920 19920 42960]
plan.type: [nan '12개월 플랜' '1개월 플랜' None]
trial.type: [nan 'A' 'B' None]


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,plan.price,plan.type,trial.type
0,3440,2023-04-05 21:49:36.156,158,570,158,11,38,34,1,4752,2,2,2
1,967,2023-03-12 08:28:29.384,158,1029,609,11,38,4,1,45697,2,2,2
2,0,2023-03-12 08:35:00.107,0,0,0,11,0,0,0,39960,2,2,2
3,3440,2023-04-12 11:36:50.250,158,570,158,11,38,34,1,143683,2,2,2
4,0,2023-02-18 04:13:20.822,0,0,0,11,0,0,0,54214,12,1,2


In [8]:
## 디코딩

for i, df in enumerate(dfs):
    print(f"file: {filelist[i]} decodings - ")
    
    # encode categorical columns using the loaded encoders
    for col in df.columns:
        # exception handling
        if col not in encoder_dict:
            continue
        if col in exclude_columns + exclude_columns_plus:
            continue
        
        # encode using the loaded encoder
        df[col] = encoder_dict[col].inverse_transform(df[col])
        print(f"{col}: {encoder_dict[col].classes_[:5]}")
    
    # display result head
    display(df.head())

file: click.cancel_plan_button.csv decodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id
0,Daegu,2023-04-10 14:57:48.556,South Korea,Windows,Windows,click.cancel_plan_button,Korean,Chrome,Web,9e59ecf9a8fbe9051bd0d54f4b702f30
1,Seoul,2023-03-12 08:26:16.224,South Korea,Apple iPad,Apple iPad,click.cancel_plan_button,English,Mobile Safari,Web,4ed4d8f12c45504a148bfff0f2e9466e
2,Uijeongbu-si,2023-03-12 08:23:04.955,South Korea,Windows,Windows,click.cancel_plan_button,Korean,Chrome,Web,b6a1bba3cc4139d7dc821561319221e6
3,Dobong-gu,2023-03-12 08:32:39.271,South Korea,Windows,Windows,click.cancel_plan_button,Korean,Chrome,Web,40acc1ddfc012f3d2f58a3de95e9e748
4,Geumjeong-gu,2023-04-12 11:36:28.311,South Korea,Windows,Windows,click.cancel_plan_button,Korean,Chrome,Web,bdffe1f18684704d4ea2d7eebcaed180


file: click.content_page_more_review_button.csv decodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
content.id: [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,content.id
0,Seo-gu,2023-04-10 14:10:01.402,South Korea,Windows,Windows,click.content_page_more_review_button,Korean,Chrome,Web,558fe7e8c06b8505f11b33d76f076e56,a1eeebeb6c307641b75b94a2a1d5b9aa
1,London,2023-01-24 21:37:43.417,United Kingdom,Windows,Windows,click.content_page_more_review_button,Korean,Whale,Web,e48956538e4df690a5d12adf1e6f2ee3,f491517f4737a60d661cd1fcacc702c0
2,Eunpyeong-gu,2023-01-31 14:44:16.897,South Korea,Windows,Windows,click.content_page_more_review_button,Korean,Chrome,Web,e745da89019e303b122bfceac0521a8f,9aa7628a347707fb155943041e2cb524
3,Seoul,2023-01-31 14:30:36.568,South Korea,Apple iPhone,Apple iPhone,click.content_page_more_review_button,Korean,Mobile Safari,Web,e2c59d444736eceee9eedba85bd4e209,f491517f4737a60d661cd1fcacc702c0
4,Seoul,2023-02-12 03:24:32.337,South Korea,Windows,Windows,click.content_page_more_review_button,Korean,Chrome,Web,74b66b738185ffdbec960db4a0da330a,101e1d0dcc38d9c86156f008a145083e


file: click.content_page_start_content_button.csv decodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
content.id: [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']
button.name: [nan '구독 시작하기' '무료로 들어보기' '무료로 시작하기' '최저

...

,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,plan.price,paid_amount,coupon.discount_amount,pg.type
0,Daejeon,2023-04-05 21:11:45.525,South Korea,Windows,Windows,resubscribe.subscription,Korean,Edge,Web,41362ad5ebcade2bb1b78344a53e7ccf,15920,15920,0,NaN
1,Seoul,2023-06-30 09:59:56.848,South Korea,Samsung Phone,Samsung Galaxy S22+,resubscribe.subscription,Korean,Chrome WebView,Web,68831c27e47f0e8f04ba809baeec218e,15920,15920,0,NaN
2,Nowon-gu,2023-06-13 06:20:22.524,South Korea,Windows,Windows,resubscribe.subscription,Korean,Chrome,Web,4e7568cb7db01f8386614ec2c6c961cd,15920,15920,0,NaN
3,Suwon,2023-06-11 11:02:11.878,South Korea,Windows,Windows,resubscribe.subscription,Korean,Chrome,Web,f91f19527b9ec688e028ec884b1b5c1c,131600,131600,0,NaN
4,Koishikawa,2023-06-13 14:39:25.736,Japan,Apple iPhone,Apple iPhone,resubscribe.subscription,Japanese,Chrome,Web,2276d7d2a31e05f4a5558c6406cc36c9,131600,131600,0,NaN


file: start.content.csv decodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
content.id: [nan '011a040b817225cb7c01d4676c21ea44' '01d9424fd9d2b1d7a4392042fc646061'
 '035f018008f1696ded6e10e1dd757726' '03b4a290c08f7fa83933827a994f2e61']
content.difficulty: [nan 'advanced' 'beginner' 'hard' 'intermediate']


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,content.id,content.difficulty
0,Daejeon,2023-04-05 21:14:39.906,South Korea,Windows,Windows,start.content,Korean,Edge,Web,41362ad5ebcade2bb1b78344a53e7ccf,445fac33232adcb3d1cdab7d8a579430,beginner
1,Seoul,2023-04-05 21:50:19.674,South Korea,Mac,Mac,start.content,Korean,Safari,Web,06b6868000a5d576180aca47896d6cce,f4be1a0fe0e301b03115591777fddf29,beginner
2,Seoul,2023-04-10 14:06:26.585,South Korea,Mac,Mac,start.content,Korean,Whale,Web,7b2ff3116db46b5e4326d49c7b027631,61b6463287573f00de13a930805a52d6,beginner
3,Toronto,2023-04-10 14:15:29.040,Canada,Windows,Windows,start.content,English,Chrome,Web,eb7515590646ead796fdfb54e2729240,61b6463287573f00de13a930805a52d6,beginner
4,Seoul,2023-04-10 14:21:36.467,South Korea,Windows,Windows,start.content,Korean,Chrome,Web,85a332649d34bc7b5ebdec7660b1d57d,18992b667be06d6f29ba0008f99d9745,advanced


file: start.free_trial.csv decodings - 
city: [nan "'Ewa Beach" '6th of October City' 'A Coruña' 'A Me']
country: [nan 'Afghanistan' 'Albania' 'Algeria' 'American Samoa']
device_family: [nan '10A30Q' '21051182G' '21061110AG' '21061119AG']
device_type: [nan 'Amazon Fire HD 10' 'Amazon Fire HD 8' 'Amazon Kindle Fire HDX'
 'Android']
event_type: ['error' 'enter.main_page' 'enter.signup_page' 'complete.signup'
 'enter.content_page']
language: [nan 'Afrikaans' 'Albanian' 'Arabic' 'Azerbaijani']
os_name: [nan 'AVG Secure Browser' 'Android Browser' 'Avast Secure Browser'
 'Chrome']
platform: [nan 'Web' None]
user_id: [nan '0000572f2ba8079bcefd2760b632e820' '000087537cbfd934375364218ff10f91'
 '000117267e1dd62481ef3ae7fb420107' '00012b6dbfccfb458ad0d08f34477944']
plan.price: [nan 14328 15920 19920 42960]
plan.type: [nan '12개월 플랜' '1개월 플랜' None]
trial.type: [nan 'A' 'B' None]


,city,client_event_time,country,device_family,device_type,event_type,language,os_name,platform,user_id,plan.price,plan.type,trial.type
0,Seoul,2023-04-05 21:49:36.156,South Korea,Mac,Mac,start.free_trial,Korean,Safari,Web,06b6868000a5d576180aca47896d6cce,15920,1개월 플랜,B
1,Dobong-gu,2023-03-12 08:28:29.384,South Korea,Windows,Windows,start.free_trial,Korean,Chrome,Web,40acc1ddfc012f3d2f58a3de95e9e748,15920,1개월 플랜,B
2,NaN,2023-03-12 08:35:00.107,NaN,NaN,NaN,start.free_trial,NaN,NaN,NaN,38aca03fba9753a6b2f31b829fb16a30,15920,1개월 플랜,B
3,Seoul,2023-04-12 11:36:50.250,South Korea,Mac,Mac,start.free_trial,Korean,Safari,Web,cc8051a75f62278b0e4216dd0a182d8c,15920,1개월 플랜,B
4,NaN,2023-02-18 04:13:20.822,NaN,NaN,NaN,start.free_trial,NaN,NaN,NaN,4cc436ad294973993ccad3e3a70053a7,131600,12개월 플랜,B
